In [1]:
import pandas as pd
import numpy as np
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split, cross_val_score,GridSearchCV
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics import mean_squared_error, r2_score ,mean_absolute_error
from flask import Flask, request, jsonify
import joblib
import warnings
import matplotlib.pyplot as plt

# Step 1:Loading Datasets

In [2]:
from sklearn.model_selection import train_test_split
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
!ls

drive  sample_data  selected_features.pkl


In [4]:
players = pd.read_csv("/content/drive/My Drive/players_22.csv") #player_22
legacy = pd.read_csv("/content/drive/My Drive/male_players.csv") #male_legacy

<ipython-input-4-13622d8f044f>:1: DtypeWarning: Columns (25,108) have mixed types. Specify dtype option on import or set low_memory=False.
  players = pd.read_csv("/content/drive/My Drive/players_22.csv") #player_22
<ipython-input-4-13622d8f044f>:2: DtypeWarning: Columns (108) have mixed types. Specify dtype option on import or set low_memory=False.
  legacy = pd.read_csv("/content/drive/My Drive/male_players.csv") #male_legacy


In [5]:
players.shape

(19239, 110)

In [6]:
legacy.shape

(161583, 110)

# Describe the datasets

In [7]:
players.describe()

,sofifa_id,overall,potential,value_eur,wage_eur,age,height_cm,weight_kg,club_team_id,league_level,...,mentality_composure,defending_marking_awareness,defending_standing_tackle,defending_sliding_tackle,goalkeeping_diving,goalkeeping_handling,goalkeeping_kicking,goalkeeping_positioning,goalkeeping_reflexes,goalkeeping_speed
count,19239.000000,19239.000000,19239.000000,1.916500e+04,19178.000000,19239.000000,19239.000000,19239.000000,19178.000000,19178.000000,...,19239.000000,19239.000000,19239.000000,19239.000000,19239.000000,19239.000000,19239.000000,19239.000000,19239.000000,2132.000000
mean,231468.086959,65.772182,71.079370,2.850452e+06,9017.989363,25.210822,181.299704,74.943032,50580.498123,1.354364,...,57.929830,46.601746,48.045584,45.906700,16.406102,16.192474,16.055356,16.229274,16.491814,36.439962
std,27039.717497,6.880232,6.086213,7.613700e+06,19470.176724,4.748235,6.863179,7.069434,54401.868535,0.747865,...,12.159326,20.200807,21.232718,20.755683,17.574028,16.839528,16.564554,17.059779,17.884833,10.751563
min,41.000000,47.000000,49.000000,9.000000e+03,500.000000,16.000000,155.000000,49.000000,1.000000,1.000000,...,12.000000,4.000000,5.000000,5.000000,2.000000,2.000000,2.000000,2.000000,2.000000,15.000000
25%,214413.500000,61.000000,67.000000,4.750000e+05,1000.000000,21.000000,176.000000,70.000000,479.000000,1.000000,...,50.000000,29.000000,28.000000,25.000000,8.000000,8.000000,8.000000,8.000000,8.000000,27.000000
50%,236543.000000,66.000000,71.000000,9.750000e+05,3000.000000,25.000000,181.000000,75.000000,1938.000000,1.000000,...,59.000000,52.000000,56.000000,53.000000,11.000000,11.000000,11.000000,11.000000,11.000000,36.000000
75%,253532.500000,70.000000,75.000000,2.000000e+06,8000.000000,29.000000,186.000000,80.000000,111139.000000,1.000000,...,66.000000,63.000000,65.000000,63.000000,14.000000,14.000000,14.000000,14.000000,14.000000,45.000000
max,264640.000000,93.000000,95.000000,1.940000e+08,350000.000000,54.000000,206.000000,110.000000,115820.000000,5.000000,...,96.000000,93.000000,93.000000,92.000000,91.000000,92.000000,93.000000,92.000000,90.000000,65.000000


In [8]:
legacy.describe()

,player_id,fifa_version,fifa_update,overall,potential,value_eur,wage_eur,age,height_cm,weight_kg,...,mentality_composure,defending_marking_awareness,defending_standing_tackle,defending_sliding_tackle,goalkeeping_diving,goalkeeping_handling,goalkeeping_kicking,goalkeeping_positioning,goalkeeping_reflexes,goalkeeping_speed
count,161583.000000,161583.000000,161583.0,161583.000000,161583.000000,1.595300e+05,159822.000000,161583.000000,161583.000000,161583.000000,...,128722.000000,161583.000000,161583.000000,161583.000000,161583.00000,161583.000000,161583.000000,161583.000000,161583.000000,17969.000000
mean,214484.722353,19.125514,2.0,65.699071,70.744008,2.326770e+06,10855.409768,25.123181,181.240205,75.235031,...,57.816892,45.757957,47.669996,45.698588,16.52961,16.274918,16.140374,16.288861,16.636973,39.149090
std,34928.608856,2.559318,0.0,7.040855,6.259121,6.005746e+06,21941.656285,4.670207,6.750148,7.000456,...,12.329739,20.453699,21.336404,20.935273,17.67047,16.834294,16.476466,16.998697,17.980143,10.503788
min,2.000000,15.000000,2.0,40.000000,40.000000,1.000000e+03,500.000000,16.000000,154.000000,49.000000,...,3.000000,1.000000,2.000000,3.000000,1.00000,1.000000,1.000000,1.000000,1.000000,12.000000
25%,199159.000000,17.000000,2.0,61.000000,66.000000,3.250000e+05,2000.000000,21.000000,176.000000,70.000000,...,50.000000,26.000000,27.000000,25.000000,8.00000,8.000000,8.000000,8.000000,8.000000,31.000000
50%,220621.000000,19.000000,2.0,66.000000,70.000000,7.250000e+05,4000.000000,25.000000,181.000000,75.000000,...,59.000000,50.000000,54.000000,52.000000,11.00000,11.000000,11.000000,11.000000,11.000000,41.000000
75%,236958.000000,21.000000,2.0,70.000000,75.000000,1.800000e+06,10000.000000,28.000000,186.000000,80.000000,...,66.000000,63.000000,66.000000,64.000000,14.00000,14.000000,14.000000,14.000000,14.000000,46.000000
max,271817.000000,23.000000,2.0,94.000000,95.000000,1.940000e+08,575000.000000,54.000000,208.000000,110.000000,...,96.000000,94.000000,94.000000,95.000000,91.00000,92.000000,95.000000,92.000000,94.000000,68.000000


# Display Information about the Dataframes

In [9]:
players.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19239 entries, 0 to 19238
Columns: 110 entries, sofifa_id to nation_flag_url
dtypes: float64(16), int64(44), object(50)
memory usage: 16.1+ MB


In [10]:
legacy.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 161583 entries, 0 to 161582
Columns: 110 entries, player_id to player_face_url
dtypes: float64(18), int64(45), object(47)
memory usage: 135.6+ MB


In [11]:
players =pd.DataFrame(players)

In [12]:
players

,sofifa_id,player_url,short_name,long_name,player_positions,overall,potential,value_eur,wage_eur,age,...,lcb,cb,rcb,rb,gk,player_face_url,club_logo_url,club_flag_url,nation_logo_url,nation_flag_url
0,158023,https://sofifa.com/player/158023/lionel-messi/...,L. Messi,Lionel Andrés Messi Cuccittini,"RW, ST, CF",93,93,78000000.0,320000.0,34,...,50+3,50+3,50+3,61+3,19+3,https://cdn.sofifa.net/players/158/023/22_120.png,https://cdn.sofifa.net/teams/73/60.png,https://cdn.sofifa.net/flags/fr.png,https://cdn.sofifa.net/teams/1369/60.png,https://cdn.sofifa.net/flags/ar.png
1,188545,https://sofifa.com/player/188545/robert-lewand...,R. Lewandowski,Robert Lewandowski,ST,92,92,119500000.0,270000.0,32,...,60+3,60+3,60+3,61+3,19+3,https://cdn.sofifa.net/players/188/545/22_120.png,https://cdn.sofifa.net/teams/21/60.png,https://cdn.sofifa.net/flags/de.png,https://cdn.sofifa.net/teams/1353/60.png,https://cdn.sofifa.net/flags/pl.png
2,20801,https://sofifa.com/player/20801/c-ronaldo-dos-...,Cristiano Ronaldo,Cristiano Ronaldo dos Santos Aveiro,"ST, LW",91,91,45000000.0,270000.0,36,...,53+3,53+3,53+3,60+3,20+3,https://cdn.sofifa.net/players/020/801/22_120.png,https://cdn.sofifa.net/teams/11/60.png,https://cdn.sofifa.net/flags/gb-eng.png,https://cdn.sofifa.net/teams/1354/60.png,https://cdn.sofifa.net/flags/pt.png
3,190871,https://sofifa.com/player/190871/neymar-da-sil...,Neymar Jr,Neymar da Silva Santos Júnior,"LW, CAM",91,91,129000000.0,270000.0,29,...,50+3,50+3,50+3,62+3,20+3,https://cdn.sofifa.net/players/190/871/22_120.png,https://cdn.sofifa.net/teams/73/60.png,https://cdn.sofifa.net/flags/fr.png,NaN,https://cdn.sofifa.net/flags/br.png
4,192985,https://sofifa.com/player/192985/kevin-de-bruy...,K. De Bruyne,Kevin De Bruyne,"CM, CAM",91,91,125500000.0,350000.0,30,...,69+3,69+3,69+3,75+3,21+3,https://cdn.sofifa.net/players/192/985/22_120.png,https://cdn.sofifa.net/teams/10/60.png,https://cdn.sofifa.net/flags/gb-eng.png,https://cdn.sofifa.net/teams/1325/60.png,https://cdn.sofifa.net/flags/be.png
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19234,261962,https://sofifa.com/player/261962/defu-song/220002,Song Defu,宋德福,CDM,47,52,70000.0,1000.0,22,...,46+2,46+2,46+2,48+2,15+2,https://cdn.sofifa.net/players/261/962/22_120.png,https://cdn.sofifa.net/teams/112541/60.png,https://cdn.sofifa.net/flags/cn.png,NaN,https://cdn.sofifa.net/flags/cn.png
19235,262040,https://sofifa.com/player/262040/caoimhin-port...,C. Porter,Caoimhin Porter,CM,47,59,110000.0,500.0,19,...,44+2,44+2,44+2,48+2,14+2,https://cdn.sofifa.net/players/262/040/22_120.png,https://cdn.sofifa.net/teams/445/60.png,https://cdn.sofifa.net/flags/ie.png,NaN,https://cdn.sofifa.net/flags/ie.png
19236,262760,https://sofifa.com/player/262760/nathan-logue/...,N. Logue,Nathan Logue-Cunningham,CM,47,55,100000.0,500.0,21,...,45+2,45+2,45+2,47+2,12+2,https://cdn.sofifa.net/players/262/760/22_120.png,https://cdn.sofifa.net/teams/111131/60.png,https://cdn.sofifa.net/flags/ie.png,NaN,https://cdn.sofifa.net/flags/ie.png
19237,262820,https://sofifa.com/player/262820/luke-rudden/2...,L. Rudden,Luke Rudden,ST,47,60,110000.0,500.0,19,...,26+2,26+2,26+2,32+2,15+2,https://cdn.sofifa.net/players/262/820/22_120.png,https://cdn.sofifa.net/teams/111131/60.png,https://cdn.sofifa.net/flags/ie.png,NaN,https://cdn.sofifa.net/flags/ie.png


In [13]:
legacy =pd.DataFrame(legacy)

In [14]:
legacy

,player_id,player_url,fifa_version,fifa_update,fifa_update_date,short_name,long_name,player_positions,overall,potential,...,cdm,rdm,rwb,lb,lcb,cb,rcb,rb,gk,player_face_url
0,158023,/player/158023/lionel-messi/150002,15,2,2014-09-18,L. Messi,Lionel Andrés Messi Cuccittini,CF,93,95,...,62+3,62+3,62+3,54+3,45+3,45+3,45+3,54+3,15+3,https://cdn.sofifa.net/players/158/023/15_120.png
1,20801,/player/20801/c-ronaldo-dos-santos-aveiro/150002,15,2,2014-09-18,Cristiano Ronaldo,Cristiano Ronaldo dos Santos Aveiro,"LW, LM",92,92,...,63+3,63+3,63+3,57+3,52+3,52+3,52+3,57+3,16+3,https://cdn.sofifa.net/players/020/801/15_120.png
2,9014,/player/9014/arjen-robben/150002,15,2,2014-09-18,A. Robben,Arjen Robben,"RM, LM, RW",90,90,...,64+3,64+3,64+3,55+3,46+3,46+3,46+3,55+3,14+3,https://cdn.sofifa.net/players/009/014/15_120.png
3,41236,/player/41236/zlatan-ibrahimovic/150002,15,2,2014-09-18,Z. Ibrahimović,Zlatan Ibrahimović,ST,90,90,...,65+3,65+3,61+3,56+3,55+3,55+3,55+3,56+3,17+3,https://cdn.sofifa.net/players/041/236/15_120.png
4,167495,/player/167495/manuel-neuer/150002,15,2,2014-09-18,M. Neuer,Manuel Peter Neuer,GK,90,90,...,40+3,40+3,36+3,36+3,38+3,38+3,38+3,36+3,87+3,https://cdn.sofifa.net/players/167/495/15_120.png
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
161578,269011,/player/269011/ziye-zhao/230002,23,2,2022-09-26,Zhao Ziye,赵梓业,RW,46,61,...,32+2,32+2,37+2,36+2,31+2,31+2,31+2,36+2,14+2,https://cdn.sofifa.net/players/269/011/23_120.png
161579,269019,/player/269019/junjie-wu/230002,23,2,2022-09-26,Wu Junjie,吴俊杰,LB,46,58,...,41+2,41+2,43+2,44+2,48+2,48+2,48+2,44+2,15+2,https://cdn.sofifa.net/players/269/019/23_120.png
161580,271093,/player/271093/kailin-barlow/230002,23,2,2022-09-26,K. Barlow,Kailin Barlow,CDM,46,58,...,45+2,45+2,47+2,47+2,46+2,46+2,46+2,47+2,14+2,https://cdn.sofifa.net/players/271/093/23_120.png
161581,271555,/player/271555/gideon-tetteh/230002,23,2,2022-09-26,G. Tetteh,Gideon Tetteh,"RW, LW",46,70,...,30+2,30+2,34+2,32+2,28+2,28+2,28+2,32+2,15+2,https://cdn.sofifa.net/players/271/555/23_120.png


In [15]:
players.dtypes

sofifa_id            int64
player_url          object
short_name          object
long_name           object
player_positions    object
                     ...  
player_face_url     object
club_logo_url       object
club_flag_url       object
nation_logo_url     object
nation_flag_url     object
Length: 110, dtype: object

In [16]:
legacy.dtypes

player_id            int64
player_url          object
fifa_version         int64
fifa_update          int64
fifa_update_date    object
                     ...  
cb                  object
rcb                 object
rb                  object
gk                  object
player_face_url     object
Length: 110, dtype: object

# Step 2: Cleaning the Data

# Dropping Unneccessary Columns in Legacy Dataset

In [17]:
legacy.columns

Index(['player_id', 'player_url', 'fifa_version', 'fifa_update',
       'fifa_update_date', 'short_name', 'long_name', 'player_positions',
       'overall', 'potential',
       ...
       'cdm', 'rdm', 'rwb', 'lb', 'lcb', 'cb', 'rcb', 'rb', 'gk',
       'player_face_url'],
      dtype='object', length=110)

In [18]:
legacy.drop(columns = ['player_id', 'player_url', 'fifa_version', 'fifa_update', 'fifa_update_date', 'player_face_url', 'short_name', 'long_name'], inplace = True)

In [19]:
legacy.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 161583 entries, 0 to 161582
Columns: 102 entries, player_positions to gk
dtypes: float64(18), int64(42), object(42)
memory usage: 125.7+ MB


# Fill missing values for numerical columns with median and for categorical columns with mode in both Datasets

In [20]:
legacy.fillna(legacy.median(numeric_only=True), inplace=True)
legacy.fillna(legacy.mode().iloc[0], inplace=True)
players.fillna(players.median(numeric_only=True), inplace=True)
players.fillna(players.mode().iloc[0], inplace=True)

# Convert relevant Columns to Categorical

In [21]:
position_columns = ['ls', 'st', 'rs', 'lw', 'lf', 'cf', 'rf', 'rw', 'lam', 'cam', 'ram',
                    'lm', 'lcm', 'cm', 'rcm', 'rm', 'lwb', 'ldm', 'cdm', 'rdm', 'rwb',
                    'lb', 'lcb', 'cb', 'rcb', 'rb', 'gk']

In [22]:
for col in position_columns:
    legacy[col] = legacy[col].astype('category')
    players[col] = players[col].astype('category')

# Step 3: Feature Extraction

In [23]:
y = legacy['overall'] #our target variable

# Get and Process all Quantitative Data

In [24]:
quant = legacy.select_dtypes(include = [np.number])

# Perform Correlation Analysis

In [25]:
corr_matrix = quant.corr()

In [26]:
corr_matrix['overall'].sort_values(ascending = False)

overall                           1.000000
movement_reactions                0.845753
potential                         0.695362
passing                           0.629868
wage_eur                          0.607297
mentality_composure               0.600033
value_eur                         0.562677
dribbling                         0.561034
attacking_short_passing           0.499757
mentality_vision                  0.493282
international_reputation          0.485234
skill_long_passing                0.483808
power_shot_power                  0.480297
physic                            0.473469
release_clause_eur                0.461524
age                               0.458390
skill_ball_control                0.457011
shooting                          0.444746
skill_curve                       0.415863
power_long_shots                  0.409086
mentality_aggression              0.398310
attacking_crossing                0.396833
skill_fk_accuracy                 0.387269
attacking_v

# Dropping columns that have a correlation less than 0.4 and greater than -0.4

In [27]:
for column in quant.columns:
  if corr_matrix['overall'][column] < 0.4 and corr_matrix['overall'][column] > -0.4:
    quant.drop(column, axis = 1, inplace = True)

In [28]:
quant.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 161583 entries, 0 to 161582
Data columns (total 20 columns):
 #   Column                    Non-Null Count   Dtype  
---  ------                    --------------   -----  
 0   overall                   161583 non-null  int64  
 1   potential                 161583 non-null  int64  
 2   value_eur                 161583 non-null  float64
 3   wage_eur                  161583 non-null  float64
 4   age                       161583 non-null  int64  
 5   international_reputation  161583 non-null  int64  
 6   release_clause_eur        161583 non-null  float64
 7   shooting                  161583 non-null  float64
 8   passing                   161583 non-null  float64
 9   dribbling                 161583 non-null  float64
 10  physic                    161583 non-null  float64
 11  attacking_short_passing   161583 non-null  int64  
 12  skill_curve               161583 non-null  int64  
 13  skill_long_passing        161583 non-null  i

In [29]:
corr_matrix = quant.corr()

In [30]:
corr_matrix['overall'].sort_values(ascending = False)

overall                     1.000000
movement_reactions          0.845753
potential                   0.695362
passing                     0.629868
wage_eur                    0.607297
mentality_composure         0.600033
value_eur                   0.562677
dribbling                   0.561034
attacking_short_passing     0.499757
mentality_vision            0.493282
international_reputation    0.485234
skill_long_passing          0.483808
power_shot_power            0.480297
physic                      0.473469
release_clause_eur          0.461524
age                         0.458390
skill_ball_control          0.457011
shooting                    0.444746
skill_curve                 0.415863
power_long_shots            0.409086
Name: overall, dtype: float64

# Imputing to fill in Missing Values

In [31]:
imputer = SimpleImputer(strategy = 'median')
q_columns = quant.columns
quant = imputer.fit_transform(quant)
quant = pd.DataFrame(quant, columns = q_columns)

In [32]:
quant

,overall,potential,value_eur,wage_eur,age,international_reputation,release_clause_eur,shooting,passing,dribbling,physic,attacking_short_passing,skill_curve,skill_long_passing,skill_ball_control,movement_reactions,power_shot_power,power_long_shots,mentality_vision,mentality_composure
0,93.0,95.0,100500000.0,550000.0,27.0,5.0,1300000.0,89.0,86.0,96.0,63.0,89.0,89.0,76.0,96.0,94.0,80.0,88.0,90.0,59.0
1,92.0,92.0,79000000.0,375000.0,29.0,5.0,1300000.0,93.0,81.0,91.0,79.0,82.0,88.0,72.0,92.0,90.0,94.0,93.0,81.0,59.0
2,90.0,90.0,54500000.0,275000.0,30.0,5.0,1300000.0,86.0,83.0,92.0,64.0,86.0,85.0,76.0,90.0,89.0,86.0,90.0,84.0,59.0
3,90.0,90.0,52500000.0,275000.0,32.0,5.0,1300000.0,91.0,81.0,86.0,86.0,84.0,80.0,76.0,90.0,85.0,93.0,88.0,83.0,59.0
4,90.0,90.0,63500000.0,300000.0,28.0,5.0,1300000.0,54.0,58.0,63.0,66.0,42.0,25.0,41.0,31.0,89.0,42.0,25.0,20.0,59.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
161578,46.0,61.0,110000.0,700.0,18.0,1.0,239000.0,48.0,40.0,47.0,44.0,38.0,39.0,33.0,45.0,39.0,64.0,31.0,42.0,40.0
161579,46.0,58.0,110000.0,750.0,19.0,1.0,193000.0,25.0,29.0,34.0,57.0,30.0,24.0,25.0,32.0,42.0,33.0,25.0,31.0,35.0
161580,46.0,58.0,110000.0,500.0,19.0,1.0,193000.0,36.0,43.0,46.0,53.0,51.0,30.0,43.0,35.0,50.0,51.0,32.0,40.0,35.0
161581,46.0,70.0,150000.0,500.0,17.0,1.0,383000.0,50.0,36.0,46.0,42.0,40.0,38.0,26.0,43.0,45.0,42.0,47.0,44.0,43.0


# Scaling the Data

In [33]:
scaler = StandardScaler()
quant = pd.DataFrame(scaler.fit_transform(quant), columns = quant.columns)
quant

,overall,potential,value_eur,wage_eur,age,international_reputation,release_clause_eur,shooting,passing,dribbling,physic,attacking_short_passing,skill_curve,skill_long_passing,skill_ball_control,movement_reactions,power_shot_power,power_long_shots,mentality_vision,mentality_composure
0,3.877514,3.875315,16.447428,24.697104,0.401872,10.148137,-0.224202,2.783978,2.926570,3.480487,-0.211839,2.086204,2.306588,1.536683,2.281310,3.535775,1.499832,2.148684,2.588430,0.085565
1,3.735485,3.396013,12.846177,16.681825,0.830120,10.148137,-0.224202,3.088467,2.421008,2.965866,1.529711,1.609554,2.251267,1.273977,2.039859,3.099571,2.401757,2.411698,1.957672,0.085565
2,3.451428,3.076478,8.742427,12.101666,1.044244,10.148137,-0.224202,2.555612,2.623233,3.068791,-0.102992,1.881925,2.085305,1.536683,1.919133,2.990520,1.886371,2.253890,2.167924,0.085565
3,3.451428,3.076478,8.407427,12.101666,1.472491,10.148137,-0.224202,2.936223,2.421008,2.451245,2.291639,1.745740,1.808701,1.536683,1.919133,2.554317,2.337334,2.148684,2.097840,0.085565
4,3.451428,3.076478,10.249927,13.246706,0.615996,10.148137,-0.224202,0.119701,0.095424,0.083989,0.114701,-1.114157,-1.233943,-0.761996,-1.642271,2.990520,-0.948252,-1.165296,-2.317466,0.085565
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
161578,-2.797832,-1.556774,-0.367900,-0.461712,-1.525244,-0.279299,-0.330265,-0.337032,-1.724598,-1.562798,-2.279930,-1.386528,-0.459452,-1.287408,-0.797192,-2.462022,0.469059,-0.849678,-0.775613,-1.639346
161579,-2.797832,-2.036076,-0.367900,-0.459422,-1.311120,-0.279299,-0.334863,-2.087843,-2.836834,-2.900813,-0.864920,-1.931270,-1.289264,-1.812820,-1.581909,-2.134869,-1.528062,-1.165296,-1.546539,-2.093270
161580,-2.797832,-2.036076,-0.367900,-0.470872,-1.311120,-0.279299,-0.334863,-1.250499,-1.421261,-1.665723,-1.300308,-0.501322,-0.957339,-0.630643,-1.400820,-1.262462,-0.368443,-0.797076,-0.915781,-2.093270
161581,-2.797832,-0.118868,-0.361200,-0.470872,-1.739368,-0.279299,-0.315870,-0.184788,-2.129048,-1.665723,-2.497623,-1.250342,-0.514773,-1.747144,-0.917918,-1.807717,-0.948252,-0.008033,-0.635445,-1.366992


In [34]:
quant.drop('overall', axis = 1, inplace = True)

# Process Categorical Data

In [35]:
cat = legacy.select_dtypes(include = ['object'])
cat.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 161583 entries, 0 to 161582
Data columns (total 15 columns):
 #   Column            Non-Null Count   Dtype 
---  ------            --------------   ----- 
 0   player_positions  161583 non-null  object
 1   dob               161583 non-null  object
 2   league_name       161583 non-null  object
 3   club_name         161583 non-null  object
 4   club_position     161583 non-null  object
 5   club_loaned_from  161583 non-null  object
 6   club_joined_date  161583 non-null  object
 7   nationality_name  161583 non-null  object
 8   nation_position   161583 non-null  object
 9   preferred_foot    161583 non-null  object
 10  work_rate         161583 non-null  object
 11  body_type         161583 non-null  object
 12  real_face         161583 non-null  object
 13  player_tags       161583 non-null  object
 14  player_traits     161583 non-null  object
dtypes: object(15)
memory usage: 18.5+ MB


In [36]:
cat = players.select_dtypes(include = ['object'])
cat.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19239 entries, 0 to 19238
Data columns (total 23 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   player_url        19239 non-null  object
 1   short_name        19239 non-null  object
 2   long_name         19239 non-null  object
 3   player_positions  19239 non-null  object
 4   dob               19239 non-null  object
 5   club_name         19239 non-null  object
 6   league_name       19239 non-null  object
 7   club_position     19239 non-null  object
 8   club_loaned_from  19239 non-null  object
 9   club_joined       19239 non-null  object
 10  nationality_name  19239 non-null  object
 11  nation_position   19239 non-null  object
 12  preferred_foot    19239 non-null  object
 13  work_rate         19239 non-null  object
 14  body_type         19239 non-null  object
 15  real_face         19239 non-null  object
 16  player_tags       19239 non-null  object
 17  player_trait

In [37]:
important_cols = ['preferred_foot', 'body_type']
cat.drop(columns = [column for column in cat.columns if column not in important_cols], inplace =True)

In [38]:
cat.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19239 entries, 0 to 19238
Data columns (total 2 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   preferred_foot  19239 non-null  object
 1   body_type       19239 non-null  object
dtypes: object(2)
memory usage: 300.7+ KB


# One hot encoding

In [39]:
cat = pd.get_dummies(cat).astype(int)
cat

,preferred_foot_Left,preferred_foot_Right,body_type_Lean (170-),body_type_Lean (170-185),body_type_Lean (185+),body_type_Normal (170-),body_type_Normal (170-185),body_type_Normal (185+),body_type_Stocky (170-),body_type_Stocky (170-185),body_type_Stocky (185+),body_type_Unique
0,1,0,0,0,0,0,0,0,0,0,0,1
1,0,1,0,0,0,0,0,0,0,0,0,1
2,0,1,0,0,0,0,0,0,0,0,0,1
3,0,1,0,0,0,0,0,0,0,0,0,1
4,0,1,0,0,0,0,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...
19234,0,1,0,1,0,0,0,0,0,0,0,0
19235,0,1,0,1,0,0,0,0,0,0,0,0
19236,0,1,0,1,0,0,0,0,0,0,0,0
19237,0,1,0,1,0,0,0,0,0,0,0,0


# Step 4:Model Selection

In [40]:
X = pd.concat([cat, quant], axis = 1)

In [41]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.8, random_state=42)

# Define and train Linear Regression

In [43]:
X_train.dropna(inplace=True)
y_train = y_train[X_train.index] # Update y_train to match the new X_train index
lr_model.fit(X_train, y_train)

LinearRegression()

# Define and train Decision Tree with GridSearchCV

In [44]:
dt_model = DecisionTreeRegressor(random_state=42)
dt_params = {
    'max_depth': [3, 5, 7],
    'min_samples_split': [2, 5],
    'min_samples_leaf': [1, 2]
}
dt_grid = GridSearchCV(dt_model, dt_params, cv=5, scoring='r2')
dt_grid.fit(X_train, y_train)

GridSearchCV(cv=5, estimator=DecisionTreeRegressor(random_state=42),
             param_grid={'max_depth': [3, 5, 7], 'min_samples_leaf': [1, 2],
                         'min_samples_split': [2, 5]},
             scoring='r2')

# Define and train Random Forest with GridSearchCV

In [45]:
rf_model = RandomForestRegressor(random_state=42)
rf_params = {
    'n_estimators': [50, 100],
    'max_depth': [3, 5],
    'min_samples_split': [5, 10],
    'min_samples_leaf': [2, 4]
}
rf_model = RandomForestRegressor(random_state=42, n_jobs=-1)
rf_grid = GridSearchCV(rf_model, rf_params, cv=5, scoring='r2', n_jobs=-1)
rf_grid.fit(X_train, y_train)

GridSearchCV(cv=5, estimator=RandomForestRegressor(n_jobs=-1, random_state=42),
             n_jobs=-1,
             param_grid={'max_depth': [3, 5], 'min_samples_leaf': [2, 4],
                         'min_samples_split': [5, 10],
                         'n_estimators': [50, 100]},
             scoring='r2')

In [46]:
gb_model = GradientBoostingRegressor(random_state=42)
gb_params = {
    'n_estimators': [50, 100],
    'max_depth': [3, 5],
    'learning_rate': [0.01, 0.1],
    'min_samples_split': [5, 10]
}
gb_grid = GridSearchCV(gb_model, gb_params, cv=5, scoring='r2', n_jobs=-1)
gb_grid.fit(X_train, y_train)

GridSearchCV(cv=5, estimator=GradientBoostingRegressor(random_state=42),
             n_jobs=-1,
             param_grid={'learning_rate': [0.01, 0.1], 'max_depth': [3, 5],
                         'min_samples_split': [5, 10],
                         'n_estimators': [50, 100]},
             scoring='r2')

# Make predictions using all models

In [49]:
# Drop rows with any missing values in X_test
X_test_dropped = X_test.dropna()

# If y_test exists, drop corresponding rows
if 'y_test' in locals():
    y_test_dropped = y_test[X_test_dropped.index]  # Align y_test with dropped X_test

# Predict using the test data with dropped rows
lr_pred = lr_model.predict(X_test_dropped)
dt_pred = dt_grid.best_estimator_.predict(X_test_dropped)
rf_pred = rf_grid.best_estimator_.predict(X_test_dropped)
gb_pred = gb_grid.best_estimator_.predict(X_test_dropped)

# Define a function to print performance metrics

In [50]:
def print_metrics(name, y_true, y_pred):
    print(f"{name}:")
    print(f"  Mean Absolute Error = {mean_absolute_error(y_true, y_pred):.4f}")
    print(f"  Mean Squared Error = {mean_squared_error(y_true, y_pred):.4f}")
    print(f"  Root Mean Squared Error = {np.sqrt(mean_squared_error(y_true, y_pred)):.4f}")
    print(f"  R2 score = {r2_score(y_true, y_pred):.4f}")
    print()

# Print performance metrics for all models

In [52]:
print_metrics("Linear Regression", y_test_dropped, lr_pred)
print_metrics("Decision Tree", y_test_dropped, dt_pred)
print_metrics("Random Forest", y_test_dropped, rf_pred)
print_metrics("Gradient Boosting", y_test_dropped, gb_pred)

Linear Regression:
  Mean Absolute Error = 1.4062
  Mean Squared Error = 3.4988
  Root Mean Squared Error = 1.8705
  R2 score = 0.9450

Decision Tree:
  Mean Absolute Error = 0.7760
  Mean Squared Error = 1.2618
  Root Mean Squared Error = 1.1233
  R2 score = 0.9802

Random Forest:
  Mean Absolute Error = 0.9293
  Mean Squared Error = 1.7251
  Root Mean Squared Error = 1.3134
  R2 score = 0.9729

Gradient Boosting:
  Mean Absolute Error = 0.5482
  Mean Squared Error = 0.5903
  Root Mean Squared Error = 0.7683
  R2 score = 0.9907



# Print best parameters for models that underwent hyperparameter tuning

In [53]:
print("Best parameters for Decision Tree:")
print(dt_grid.best_params_)
print("\nBest parameters for Random Forest:")
print(rf_grid.best_params_)
print("\nBest parameters for Gradient Boosting:")
print(gb_grid.best_params_)

Best parameters for Decision Tree:
{'max_depth': 7, 'min_samples_leaf': 2, 'min_samples_split': 5}

Best parameters for Random Forest:
{'max_depth': 5, 'min_samples_leaf': 2, 'min_samples_split': 5, 'n_estimators': 100}

Best parameters for Gradient Boosting:
{'learning_rate': 0.1, 'max_depth': 5, 'min_samples_split': 5, 'n_estimators': 100}


# Compare the best scores from GridSearchCV

In [54]:
print("\nBest cross-validation scores:")
print(f"Decision Tree: {dt_grid.best_score_:.4f}")
print(f"Random Forest: {rf_grid.best_score_:.4f}")
print(f"Gradient Boosting: {gb_grid.best_score_:.4f}")


Best cross-validation scores:
Decision Tree: 0.9790
Random Forest: 0.9745
Gradient Boosting: 0.9904


 **Test Using Player's Data**

In [55]:
print("All columns in the legacy dataset:")
print(legacy.columns.tolist())

All columns in the legacy dataset:
['player_positions', 'overall', 'potential', 'value_eur', 'wage_eur', 'age', 'dob', 'height_cm', 'weight_kg', 'league_id', 'league_name', 'league_level', 'club_team_id', 'club_name', 'club_position', 'club_jersey_number', 'club_loaned_from', 'club_joined_date', 'club_contract_valid_until_year', 'nationality_id', 'nationality_name', 'nation_team_id', 'nation_position', 'nation_jersey_number', 'preferred_foot', 'weak_foot', 'skill_moves', 'international_reputation', 'work_rate', 'body_type', 'real_face', 'release_clause_eur', 'player_tags', 'player_traits', 'pace', 'shooting', 'passing', 'dribbling', 'defending', 'physic', 'attacking_crossing', 'attacking_finishing', 'attacking_heading_accuracy', 'attacking_short_passing', 'attacking_volleys', 'skill_dribbling', 'skill_curve', 'skill_fk_accuracy', 'skill_long_passing', 'skill_ball_control', 'movement_acceleration', 'movement_sprint_speed', 'movement_agility', 'movement_reactions', 'movement_balance', 'p

In [64]:
columns_to_drop = ['league_id', 'club_joined_date', 'club_contract_valid_until_year']

In [65]:
z = legacy['overall']

In [66]:
feature_columns = [col for col in legacy.columns if col not in columns_to_drop]

In [67]:
print("\nSelected feature columns:")
print(feature_columns)


Selected feature columns:
['player_positions', 'overall', 'potential', 'value_eur', 'wage_eur', 'age', 'dob', 'height_cm', 'weight_kg', 'league_name', 'league_level', 'club_team_id', 'club_name', 'club_position', 'club_jersey_number', 'club_loaned_from', 'nationality_id', 'nationality_name', 'nation_team_id', 'nation_position', 'nation_jersey_number', 'preferred_foot', 'weak_foot', 'skill_moves', 'international_reputation', 'work_rate', 'body_type', 'real_face', 'release_clause_eur', 'player_tags', 'player_traits', 'pace', 'shooting', 'passing', 'dribbling', 'defending', 'physic', 'attacking_crossing', 'attacking_finishing', 'attacking_heading_accuracy', 'attacking_short_passing', 'attacking_volleys', 'skill_dribbling', 'skill_curve', 'skill_fk_accuracy', 'skill_long_passing', 'skill_ball_control', 'movement_acceleration', 'movement_sprint_speed', 'movement_agility', 'movement_reactions', 'movement_balance', 'power_shot_power', 'power_jumping', 'power_stamina', 'power_strength', 'powe

In [68]:
X = legacy[feature_columns]
y = legacy['overall'] # Use the column name 'overall' directly

In [69]:
print(f"\nShape of feature matrix X: {X.shape}")
print(f"Shape of target variable y: {y.shape}")


Shape of feature matrix X: (161583, 99)
Shape of target variable y: (161583,)


In [70]:
import joblib
joblib.dump(feature_columns, 'selected_features.pkl')

['selected_features.pkl']

In [71]:
print("\nFeature columns have been saved to 'selected_features.pkl'")


Feature columns have been saved to 'selected_features.pkl'


In [72]:
# Load the saved feature columns
feature_columns = joblib.load('selected_features.pkl')

# Use these features for the players_22 dataset
X_22 = players[feature_columns]
y_22 = players['overall']

In [102]:
# When initially scaled your training data:
scaler = StandardScaler()

# Select only numeric columns for scaling
numeric_columns = X.select_dtypes(include=['number']).columns
X_numeric = X[numeric_columns]

X_scaled = pd.DataFrame(scaler.fit_transform(X_numeric), columns=X_numeric.columns)

# Saving the Scaler
import joblib
joblib.dump(scaler, 'scaler.pkl')

scaler = joblib.load('scaler.pkl')

# Select numeric columns in X_22
X_22_numeric = X_22[numeric_columns]

X_22_scaled = pd.DataFrame(scaler.transform(X_22_numeric), columns=X_22_numeric.columns)

 Step 6: Deploy the Model with Flask

In [90]:
best_model = gb_grid.best_estimator_
joblib.dump(best_model, 'fifa_rating_predictor.joblib')

['fifa_rating_predictor.joblib']

In [103]:
from google.colab import files
files.download('fifa_rating_predictor.joblib')


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [99]:
app = Flask(__name__)

@app.route('/predict', methods=['POST'])
def predict():
    data = request.json
    features = pd.DataFrame(data, index=[0])
    prediction = best_model.predict(features)
    return jsonify({'predicted_rating': prediction[0]})

if __name__ == '__main__':
    app.run(debug=False)

 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit
